In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pathlib import Path
import panel as pn
from panel.interact import interact
import plotly.express as px
pn.extension("plotly")

%matplotlib inline


Bad key savefig.frameon in file C:\Users\William Riera\.conda\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\William Riera\.conda\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\William Riera\.conda\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stde

In [2]:
# Load .env environment variables
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_api_key")

In [4]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
# Format start and end dates as ISO format for our bond indices
tickers = ['GOLD', 'WFG', 'IP', 'ASA', 'CMC', 'GLHA']
start_date = pd.Timestamp("1990-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-04-20", tz="America/New_York").isoformat()

commodities_portfolio = alpaca.get_bars(
    tickers,
    timeframe="1Day",
    start = start_date,
    end = end_date
).df

In [6]:
# sets the index as a dateformat
commodities_portfolio.index = commodities_portfolio.index.date
commodities_portfolio.head()

,open,high,low,close,volume,trade_count,vwap,symbol
2015-12-01,7.18,7.4100,6.8803,7.38,97337,310,7.262090,ASA
2015-12-02,7.24,7.4399,7.1400,7.27,96037,251,7.266087,ASA
2015-12-03,7.35,7.4900,7.2400,7.40,95499,375,7.398886,ASA
2015-12-04,7.49,7.7500,7.4400,7.74,161779,400,7.619966,ASA
2015-12-07,7.64,7.6500,7.2600,7.33,137773,428,7.452271,ASA


In [7]:
#reformats the data
commodities_portfolio = commodities_portfolio.drop(columns=['open','high','low','volume','trade_count','vwap'])
GOLD_closes = commodities_portfolio[commodities_portfolio['symbol']=='GOLD'].drop('symbol', axis=1)
WFG_closes = commodities_portfolio[commodities_portfolio['symbol']=='WFG'].drop('symbol', axis=1)
IP_closes = commodities_portfolio[commodities_portfolio['symbol']=='IP'].drop('symbol', axis=1)
ASA_closes = commodities_portfolio[commodities_portfolio['symbol']=='ASA'].drop('symbol', axis=1)
CMC_closes = commodities_portfolio[commodities_portfolio['symbol']=='CMC'].drop('symbol', axis=1)
GLHA_closes = commodities_portfolio[commodities_portfolio['symbol']=='GLHA'].drop('symbol', axis=1)
commodities_portfolio_close = pd.concat([GOLD_closes, WFG_closes, IP_closes, ASA_closes,
CMC_closes, GLHA_closes],axis=1, keys=['GOLD', 'WFG', 'IP', 'ASA', 'CMC', 'GLHA'])

commodities_index_all = pd.DataFrame()
commodities_index_all["GOLD"] = commodities_portfolio_close["GOLD"]["close"]
commodities_index_all["WFG"] = commodities_portfolio_close["WFG"]["close"]
commodities_index_all["IP"] =  commodities_portfolio_close["IP"]["close"]
commodities_index_all["ASA"] =  commodities_portfolio_close["ASA"]["close"]
commodities_index_all["CMC"] =  commodities_portfolio_close["CMC"]["close"]
commodities_index_all["GLHA"] =  commodities_portfolio_close["GLHA"]["close"]
commodities_index_all.head()



,GOLD,WFG,IP,ASA,CMC,GLHA
2015-12-01,63.45,NaN,41.22,7.38,15.02,NaN
2015-12-02,62.71,NaN,40.07,7.27,14.82,NaN
2015-12-03,63.03,NaN,39.61,7.40,14.62,NaN
2015-12-04,65.18,NaN,40.23,7.74,14.81,NaN
2015-12-07,62.67,NaN,39.78,7.33,14.44,NaN


In [8]:
#csvpath = Path('C:/Users/15613/FinTechWork/FinTechProject1/BondIndexData.csv')
#bond_index_all.to_csv(csvpath)

In [9]:
#creates a selection widget
bond_tickers = pn.widgets.MultiChoice(name="Security List", value=['GOLD', 'WFG', 'IP', 'ASA', 'CMC', 'GLHA'],
    options=tickers)
pn.Column(bond_tickers, height=200)

BokehModel(combine_events=True, render_bundle={'docs_json': {'99f398ac-e25f-4ef6-bfee-166c0560a6a5': {'defs': …

Column(height=200)
    [0] MultiChoice(name='Security List', options=['GOLD', 'WFG', ...], value=['GOLD', 'WFG', ...])

In [10]:
#creates a new dataframe from the widget selection
selected_commodities_indexes = commodities_index_all[bond_tickers.value]
selected_commodities_indexes

,GOLD,WFG,IP,ASA,CMC,GLHA
2015-12-01,63.45,NaN,41.22,7.38,15.02,NaN
2015-12-02,62.71,NaN,40.07,7.27,14.82,NaN
2015-12-03,63.03,NaN,39.61,7.40,14.62,NaN
2015-12-04,65.18,NaN,40.23,7.74,14.81,NaN
2015-12-07,62.67,NaN,39.78,7.33,14.44,NaN
...,...,...,...,...,...,...
2022-04-13,25.58,78.29,47.05,22.85,43.72,9.750
2022-04-14,25.20,77.44,47.43,22.88,44.32,9.765
2022-04-18,25.38,78.26,47.46,22.97,43.66,9.785
2022-04-19,25.01,80.22,48.65,22.67,45.05,9.800
